
# 0. Module 0: Prerequisites and Setup

This module ensures students have the necessary technical and theoretical background.

---

## 0.1 Technical Prerequisite: Python Foundations

A core component of quantitative research is the ability to handle and process large financial datasets efficiently. We rely heavily on the following libraries:

1. **Pandas:** For managing time-series data using DataFrames.  
2. **NumPy:** For high-performance numerical operations and vectorization.  
3. **Matplotlib/Seaborn:** For data visualization.

### Review Exercise: Basic Data Manipulation


In [ ]:

import pandas as pd
import numpy as np

# Creating a basic DataFrame
dates = pd.date_range('2024-01-01', periods=5)
data = {'Price': [100, 102, 99, 105, 104], 'Volume': [1000, 1500, 800, 2000, 1200]}
df_ex = pd.DataFrame(data, index=dates)

print("Sample DataFrame:")
print(df_ex)

# Quick review of calculating log returns and basic stats
log_returns_ex = np.log(df_ex['Price'] / df_ex['Price'].shift(1)).dropna()

print("\nLog Returns (first 4):")
print(log_returns_ex)
print(f"\nMean Return: {log_returns_ex.mean():.4f}")
print(f"Standard Deviation (Risk): {log_returns_ex.std():.4f}")



## 0.2 Theoretical Prerequisite: Probability and Statistics

Risk measurement relies fundamentally on understanding the distribution of returns.

### Key Concepts to Recall

- **Mean (μ):** The average return; our estimate of expected return.  
- **Variance (σ²) & Standard Deviation (σ):** The spread of returns around the mean, defining volatility (risk).  
- **Skewness:** Measures the asymmetry of the distribution (negative skew is bad, indicating frequent small gains but rare large losses).  
- **Kurtosis:** Measures the "tailedness" or "peakedness" of the distribution. High (excess) kurtosis indicates fat tails, meaning extreme events are more likely than predicted by the Normal Distribution.

### Review Exercise: QQ-Plots and Normality

The Quantile-Quantile (QQ) Plot is the primary visual tool to check if a dataset follows a normal distribution. 
If the data points hug the 45-degree line, the assumption holds.


In [ ]:

import matplotlib.pyplot as plt
import scipy.stats as stats

# Use the log returns from the previous exercise
np.random.seed(42)
normal_sample = np.random.normal(0, log_returns_ex.std(), 100)

plt.figure(figsize=(10, 5))
stats.probplot(normal_sample, dist="norm", plot=plt)
plt.title("QQ-Plot: Normal Sample (Expected)")
plt.show()



# Week 0: Advanced Risk Analysis Lecture  
### Using Python, yfinance, and Statistical Foundations  

This lecture introduces key concepts in **financial risk measurement** and **quantitative portfolio analysis**.  
By the end of this session, students will understand:

- The meaning of *financial risk* and its different forms (systematic vs idiosyncratic)
- How to compute and interpret *volatility*, *covariance*, and *risk-return ratios*
- How to use the **yfinance** API to pull real asset data for empirical tests
- Applications of *Value at Risk (VaR)* and *Conditional VaR (CVaR)*
- Practical coding skills for **portfolio diversification** and **tail risk management**

> **Tip:** Run all cells in order to ensure that data dependencies and plots render correctly.


# Week 0 — Advanced Risk Analysis (Lecture-style)

This notebook is a lecture-style, in-depth Week 0 module for an advanced course in Risk Analysis. It uses real market data via `yfinance` to demonstrate key concepts. Run the cells top-to-bottom in Jupyter.

**Contents **:
- Intro: why risk matters
- Statistical foundations (LLN, CLT, tails)
- Volatility and risk-return metrics
- Portfolio risk: covariance, diversification, beta
- Tail risk: VaR, CVaR, EVT intro
- Practical labs using `yfinance` (data fetch, computations, plots)
- Exercises & further reading

**Note:** This notebook fetches data using `yfinance`. Ensure you have internet access when running the fetch cells.

## 1. The Nature of Financial Risk — Why this matters

- Financial markets are driven by uncertainty. Risk measurement underpins portfolio construction, risk limits, and regulatory capital.

**Lecture note:** Real-world risk is not just volatility — it includes liquidity, credit, and systemic components. Throughout this course we focus on **market & tail risk** and tools to measure them.

**Case study (motivation):** 2008 Global Financial Crisis — common risk measures failed because of hidden correlations, leverage, and extreme tail events. We'll see why classical Gaussian models mislead risk managers.



## Statistical Foundations: LLN & CLT

The **Law of Large Numbers (LLN)** states that:
\[
\bar{X}_n = \frac{1}{n}\sum_{i=1}^n X_i \xrightarrow{p} \mu
\]
as \( n \to \infty \). This implies that sample averages converge to the population mean.

The **Central Limit Theorem (CLT)** states:
\[
\sqrt{n}(\bar{X}_n - \mu) \xrightarrow{d} N(0, \sigma^2)
\]
meaning the standardized sample mean converges in distribution to a normal distribution.

However, in financial data:
- Returns exhibit **skewness** and **excess kurtosis**.
- This violates the normality assumption underlying CLT-based risk models.
- Heavy tails imply larger-than-expected extreme events (e.g., crashes).

We observe this empirically through QQ-plots and tail estimators.


## 2. Data Fetch (yfinance)

This notebook will use daily prices for several tickers (e.g., SPY, AAPL, MSFT, GLD, TLT). Run the cell below to fetch data. If `yfinance` is not installed, install with `pip install yfinance`.

**Important:** If you are behind a firewall, ensure your environment can access Yahoo Finance.

In [ ]:

# Why use log returns?
# Log returns are additive over time (ln(P_t/P_{t-1})) and simplify compounding.
# They also stabilize variance and are more appropriate for statistical modeling.
# In financial econometrics, they are preferred for aggregation and for applying CLT-based tools.


In [ ]:
# Data fetch using yfinance
import yfinance as yf
import pandas as pd

tickers = ['SPY', 'AAPL', 'MSFT', 'GLD', 'TLT']
start = '2015-01-01'
end = None

# Fetch adjusted close prices
data = yf.download(tickers, start=start, end=end, progress=False)['Adj Close']
data = data.dropna()
data.head()

## 3. Returns, Log-returns, and Summary Statistics

Compute simple returns and log-returns. We show mean, volatility, skewness, and kurtosis. Lecture note: use log-returns for aggregation over time; simple returns are intuitive for percentage changes.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import skew, kurtosis

# Compute daily simple returns and log returns
returns = data.pct_change().dropna()
log_returns = np.log(data).diff().dropna()

# Summary statistics
stats = returns.describe().T
stats['skew'] = returns.apply(skew)
stats['kurtosis'] = returns.apply(kurtosis)
stats['annualized_vol'] = returns.std() * (252**0.5)
stats[['mean','annualized_vol','skew','kurtosis']].round(4)

## 4. Visuals: Price Series, Return Distributions, QQ-plots

Visual intuition is crucial. Below we plot price series, rolling volatility, histograms, and QQ-plots to inspect normality and tails.

In [ ]:

# Hill estimator intuition:
# The Hill estimator is used to estimate the tail index of heavy-tailed distributions.
# A smaller tail index implies fatter tails — more extreme risk events.
# This helps in understanding how much worse than "normal" the extreme outcomes can be.
# It's particularly useful for stress testing and systemic risk measurement.


In [ ]:
import seaborn as sns
import statsmodels.api as sm

plt.style.use('seaborn-darkgrid')
data.plot(title='Adjusted Close Prices', figsize=(12,6))
plt.show()

# Rolling volatility
rolling_vol = returns['SPY'].rolling(window=21).std() * (252**0.5)
rolling_vol.plot(title='21-day Rolling Annualized Volatility (SPY)')
plt.show()

# Histogram + QQ for SPY returns
plt.figure(figsize=(12,4))
plt.subplot(1,2,1)
sns.histplot(returns['SPY'], bins=80, kde=True)
plt.title('SPY Daily Returns Histogram')

plt.subplot(1,2,2)
sm.qqplot(returns['SPY'], line='s')
plt.title('QQ-plot SPY vs Normal')
plt.tight_layout()
plt.show()

## 5. Volatility, Sharpe Ratio, and Drawdowns

Definitions and practical computation. We compute portfolio metrics for an equally-weighted portfolio of the tickers.

In [ ]:
# Portfolio equal-weight example
weights = np.repeat(1/len(tickers), len(tickers))
portfolio_returns = returns.dot(weights)

annualized_return = (1+portfolio_returns.mean())**252 - 1
annualized_vol = portfolio_returns.std() * (252**0.5)
sharpe = (annualized_return - 0.01) / annualized_vol  # assume 1% rf

# Drawdown computation
cum_returns = (1 + portfolio_returns).cumprod()
running_max = cum_returns.cummax()
drawdown = (cum_returns - running_max) / running_max

print(f"Annualized Return: {annualized_return:.3%}")
print(f"Annualized Volatility: {annualized_vol:.3%}")
print(f"Sharpe Ratio (rf=1%): {sharpe:.2f}")
print(f"Max Drawdown: {drawdown.min():.2%}")

cum_returns.plot(title='Portfolio Cumulative Returns', figsize=(12,5))
drawdown.plot(title='Portfolio Drawdown', figsize=(12,4), color='red')
plt.show()

## 6. Portfolio Variance and Covariance Matrix

Derive portfolio variance: \(\sigma_p^2 = w^\top \Sigma w\). We display covariance and correlation heatmaps.

In [ ]:

# VaR (Value-at-Risk) and ES (Expected Shortfall) Intuition:
# VaR answers: "What is the worst expected loss at a given confidence level (e.g., 95%)?"
# ES answers: "If we exceed VaR, how large are the average losses beyond that point?"
# These are crucial for capital allocation, risk management, and regulatory reporting.


In [ ]:
cov = returns.cov()
corr = returns.corr()

import seaborn as sns
plt.figure(figsize=(8,6))
sns.heatmap(corr, annot=True, cmap='vlag')
plt.title('Correlation Matrix')
plt.show()

## 7. Beta Estimation & CAPM

Estimate beta for each asset relative to SPY using OLS. Discuss interpretation: beta as sensitivity to market.

In [ ]:
import statsmodels.api as sm

market = returns['SPY']
betas = {}
for col in returns.columns:
    if col == 'SPY':
        continue
    y = returns[col]
    X = sm.add_constant(market)
    res = sm.OLS(y, X).fit()
    betas[col] = res.params[1]
betas

## 8. Value at Risk (VaR) and Conditional VaR (CVaR)

Compute historical VaR, parametric (Gaussian) VaR, and CVaR for portfolio. Discuss pros/cons.

In [ ]:
import numpy as np

alpha = 0.01  # 99% VaR level as lower tail
# Historical VaR
VaR_hist = -np.quantile(portfolio_returns, alpha)
# Parametric VaR (Gaussian)
mu = portfolio_returns.mean()
sigma = portfolio_returns.std()
z = np.quantile(np.random.normal(size=1000000), alpha)
VaR_param = -(mu + z * sigma)
# CVaR (Expected Shortfall) historical
losses = -portfolio_returns
CVaR_hist = losses[losses >= VaR_hist].mean()

print(f"Historical VaR (99%): {VaR_hist:.4%}")
print(f"Parametric VaR (99%): {VaR_param:.4%}")
print(f"Historical CVaR (99%): {CVaR_hist:.4%}")

## 9. Backtesting VaR (Kupiec Test)

Implement a simple Kupiec proportion-of-failures test to see whether VaR breaches match expectation.

In [ ]:
from scipy.stats import binom_test

# Kupiec test (unconditional coverage)
breaches = (portfolio_returns < -VaR_hist).sum()
n_obs = len(portfolio_returns)
p_hat = breaches / n_obs
p0 = alpha

print(f"Breaches: {breaches} / {n_obs} (observed rate={p_hat:.4f})")
# two-sided binomial test (exact)
pvalue = binom_test(breaches, n_obs, p0, alternative='two-sided')
print(f"Kupiec (binomial) p-value: {pvalue:.4f}")

## 10. Tail Analysis: QQ-plots and Intro to EVT

Use QQ-plots and empirical tail index plots. Introduce Peaks Over Threshold and GPD for later weeks.

In [ ]:
import numpy as np
from statsmodels.graphics.gofplots import qqplot

# QQ-plot for portfolio returns
qqplot(portfolio_returns, line='s')
plt.title('QQ-plot: Portfolio Returns vs Normal')
plt.show()

# Empirical tail index (Hill-ish) simple diagnostic for SPY
sorted_losses = np.sort(-returns['SPY'])[::-1]  # descending losses
k = 500
hill_est = np.mean(np.log(sorted_losses[:k]) - np.log(sorted_losses[k]))
print(f"Rough Hill estimate (k={k}): {1/hill_est if hill_est>0 else np.nan:.3f}")

## 11. Scenario Analysis & Stress Testing

Demonstrate stress scenarios: sudden volatility spike; correlation going to 0.9; market crash scenario.

In [ ]:
# Stress scenario: market crash and correlation spike
# 1) Market crash: -10% on SPY day (shock)
shock = -0.10
# approximate impact on portfolio: revalue by applying same shock proportionally to holdings via betas
approx_port_loss = sum([betas.get(ticker, 1.0) * shock * w for ticker, w in zip(returns.columns, [1/len(returns.columns)]*len(returns.columns))])
print(f"Approx portfolio loss from -10% market shock (beta-weighted): {approx_port_loss:.2%}")

# 2) Correlation shock: recompute portfolio volatility with high correlation
high_corr = corr.copy()
high_corr_values = high_corr.values
high_corr_values[high_corr_values<1] = 0.9  # set off-diagonals to 0.9 roughly
# rebuild covariance assuming same vol but high corr (simplified)
vols = returns.std().values
cov_high = np.outer(vols, vols) * 0.9
w = np.repeat(1/len(vols), len(vols))
sigma_p_high = (w @ cov_high @ w)**0.5
print(f"Portfolio vol under high-correlation scenario: {sigma_p_high* (252**0.5):.2%} annualized")

## 12. Visualization Summary

Key visual aids included: price series, rolling volatility, histograms, QQ-plots, correlation heatmap, drawdowns. These are essential for communicating risk to stakeholders.

## 13. Exercises (Detailed)

1. Compute parametric VaR and historical VaR for each individual asset and compare which assets are underestimated by parametric VaR.

2. Implement bootstrapped confidence intervals for the historical VaR of the portfolio (95% CI).

3. Fit a t-distribution to SPY returns and compare 99.9% quantile to Gaussian.

4. Create a function that shocks the correlation matrix and find the worst-case VaR subject to the covariance matrix remaining positive semidefinite.

5. (Advanced) Implement POT/GPD fit for SPY returns and compute 1-day 99.9% VaR using EVT.

## 14. Further Reading & References

- McNeil, Frey & Embrechts — *Quantitative Risk Management* (Chapters on VaR, EVT, Copulas)
- Embrechts et al. — *Modelling Extremal Events*
- Cont — *Empirical properties of asset returns*
- Rockafellar & Uryasev — *Optimization of CVaR*

---
End of Week 0 lecture notebook. Move to Week 1 for Portfolio Risk & Correlations.

In [ ]:

import numpy as np
from ipywidgets import interact
import matplotlib.pyplot as plt

# Simulated returns as placeholder
data = np.random.normal(0, 0.02, 1000)

@interact(alpha=(0.90, 0.99, 0.999))
def show_var(alpha=0.95):
    var_est = -np.quantile(data, 1 - alpha)
    plt.hist(data, bins=40, color='lightblue', edgecolor='k')
    plt.axvline(-var_est, color='red', linestyle='--', label=f'VaR@{alpha*100:.1f}%')
    plt.legend()
    plt.title(f'Interactive VaR Visualization (alpha={alpha:.3f})')
    plt.show()



# Summary and Discussion Points

This lecture demonstrated how risk measures bridge theory and practice. Recall the following:

1. **Risk and Return** are two sides of the same coin — expected return compensates for uncertainty.
2. **Volatility** captures dispersion, but not direction or tail heaviness.
3. **Covariance Structure** guides portfolio construction; diversification is not elimination of risk, but management of exposure.
4. **Tail Metrics (VaR/CVaR)** are essential under non-Gaussian distributions to anticipate extreme losses.

---
### Real-World Extensions

- Implement rolling-window VaR/CVaR to explore time-varying risk.
- Compare historical VaR estimates to parametric ones.
- Integrate macroeconomic indicators (e.g., VIX, interest rates) for a multi-factor risk model.

---
**Exercise:** Download price series for three additional ETFs and compute portfolio VaR using your own weighting scheme. Discuss how diversification affects tail risk.



## Enhanced VaR & CVaR: Student's t and Gaussian CVaR

**Motivation:** The Gaussian VaR often underestimates extreme losses when returns exhibit heavy tails.
We add a Student's *t* parametric VaR (fit to data) and an analytical Gaussian CVaR to compare with Historical estimates.

**What students should learn:**  
- How to fit a Student's *t* distribution to returns and compute the corresponding VaR.  
- How Gaussian analytical CVaR differs from empirical CVaR and why it can understate tail risk.


In [ ]:

# --- t-VaR and Gaussian CVaR calculation ---
# This cell assumes `portfolio_returns` (daily simple returns) is already defined in the notebook.
# If not, it will fetch data for an equally-weighted portfolio of example tickers.

import numpy as np
import scipy.stats as st
import yfinance as yf

try:
    portfolio_returns  # check if exists
except NameError:
    # recreate a simple portfolio if variable not present
    tickers = ['SPY','AAPL','MSFT','GLD','TLT']
    data = yf.download(tickers, start='2015-01-01', progress=False)['Adj Close'].dropna()
    returns = data.pct_change().dropna()
    weights = np.repeat(1/len(tickers), len(tickers))
    portfolio_returns = returns.dot(weights)

alpha = 0.01  # 99% VaR (lower tail alpha)
mu = portfolio_returns.mean()
sigma = portfolio_returns.std(ddof=1)

# Historical VaR/ES
VaR_hist = -np.quantile(portfolio_returns, alpha)
ES_hist = -portfolio_returns[portfolio_returns <= np.quantile(portfolio_returns, alpha)].mean()

# Gaussian parametric VaR/ES (analytical CVaR)
z = st.norm.ppf(alpha)
VaR_gauss = -(mu + z * sigma)
ES_gauss = -(mu + sigma * st.norm.pdf(z) / (1 - alpha))

# Fit Student's t to portfolio returns (use scipy.stats.t.fit)
# scipy returns (df, loc, scale)
t_params = st.t.fit(portfolio_returns)
df_t, loc_t, scale_t = t_params
# t-VaR (use ppf)
VaR_t = -st.t.ppf(alpha, df_t, loc=loc_t, scale=scale_t)
# For t-ES (expected shortfall) we can approximate by numerical integration if desired
# Here compute empirical ES under fitted t by sampling a large number of points
rng = np.random.default_rng(12345)
samples_t = st.t.rvs(df_t, loc=loc_t, scale=scale_t, size=200000, random_state=rng)
ES_t_approx = -np.mean(samples_t[samples_t <= np.quantile(samples_t, alpha)])

print(f"Data points used: {len(portfolio_returns)}")
print(f"Historical VaR (99%): {VaR_hist:.6f}, ES (empirical): {ES_hist:.6f}")
print(f"Gaussian VaR (99%): {VaR_gauss:.6f}, Gaussian analytical ES: {ES_gauss:.6f}")
print(f"Student-t fit params: df={df_t:.3f}, loc={loc_t:.6f}, scale={scale_t:.6f}")
print(f"t-VaR (99%): {VaR_t:.6f}, t-ES approx (by sampling): {ES_t_approx:.6f}")


## Systematic vs Idiosyncratic Risk Decomposition

After estimating beta via OLS (CAPM regression), we can decompose an asset's total variance into:
\[
\sigma_{\text{Total}}^2 = \sigma_{\text{Systematic}}^2 + \sigma_{\text{Idiosyncratic}}^2
\]
where
\[
\sigma_{\text{Systematic}}^2 = \beta^2 \sigma_{\text{Market}}^2
\]

We'll compute this decomposition for one asset (AAPL) and visualize the split.


In [ ]:

# --- Systematic vs Idiosyncratic variance decomposition ---
import statsmodels.api as sm
import matplotlib.pyplot as plt

# Ensure returns DataFrame exists; if not, fetch sample
try:
    returns  # should be a DataFrame of asset returns
except NameError:
    tickers = ['SPY','AAPL','MSFT','GLD','TLT']
    data = yf.download(tickers, start='2015-01-01', progress=False)['Adj Close'].dropna()
    returns = data.pct_change().dropna()

market = returns['SPY']
asset = returns['AAPL']

# OLS regression: asset ~ market
X = sm.add_constant(market)
res = sm.OLS(asset, X).fit()
beta = res.params[1]
r2 = res.rsquared

sigma_market = market.std(ddof=1)
sigma_asset = asset.std(ddof=1)

sigma_systematic_sq = beta**2 * sigma_market**2
sigma_idio_sq = sigma_asset**2 - sigma_systematic_sq

print(res.summary().as_text())
print('\nDecomposition:')
print(f'beta = {beta:.4f}, R-squared = {r2:.4f}')
print(f'total variance = {sigma_asset**2:.8f}')
print(f'systematic variance = {sigma_systematic_sq:.8f}')
print(f'idiosyncratic variance = {sigma_idio_sq:.8f}')

# Visualize decomposition
labels = ['Systematic', 'Idiosyncratic']
vals = [sigma_systematic_sq, sigma_idio_sq]
plt.figure(figsize=(6,4))
plt.bar(labels, vals, color=['#4c72b0','#dd8452'])
plt.title('Variance Decomposition for AAPL (daily variance)')
plt.ylabel('Variance (daily)')
plt.show()


##  Hill Estimator & Hill Plot

The Hill estimator for the tail index depends on the choice of `k` (number of top order statistics).  
A **Hill plot** shows the estimated tail index as a function of `k`. Look for ranges where the estimate stabilizes — those are plausible choices for `k`.

We'll plot the Hill estimate for SPY losses and annotate the plot.


In [ ]:

# --- Hill plot for SPY losses ---
import numpy as np
import matplotlib.pyplot as plt

# Use negative returns (losses) for tail analysis
try:
    spy_losses = -returns['SPY'].dropna().values
except NameError:
    data = yf.download('SPY', start='2015-01-01', progress=False)['Adj Close'].dropna()
    spy_returns = data.pct_change().dropna()
    spy_losses = -spy_returns.values

# Sort descending (largest losses first)
sorted_losses = np.sort(spy_losses)[::-1]
n = len(sorted_losses)
ks = np.arange(10, min(2000, n//2), 10)

hill_estimates = []
for k in ks:
    top_k = sorted_losses[:k]
    threshold = sorted_losses[k]
    # Hill estimator: 1/alpha_hat = (1/k) * sum_{i=1}^k log(X_i / X_{k+1})
    logs = np.log(top_k / threshold)
    hill_inv = np.mean(logs)
    tail_index = 1 / hill_inv if hill_inv > 0 else np.nan
    hill_estimates.append(tail_index)

plt.figure(figsize=(10,5))
plt.plot(ks, hill_estimates, marker='o', markersize=3)
plt.xlabel('k (number of top order statistics)')
plt.ylabel('Estimated tail index (alpha)')
plt.title('Hill Plot (SPY losses) — look for stable regions')
plt.grid(True)
plt.show()


##  Dynamic Risk-Free Rate via BIL ETF (proxy)

Rather than hardcoding a risk-free rate, we fetch **BIL** (a 3-month T-bill ETF) and compute the realized annualized return over the sample window. We then use that rate to compute the Sharpe ratio more realistically.


In [ ]:

# --- Fetch BIL and compute realized risk-free rate ---
import yfinance as yf
import numpy as np

start = returns.index[0].strftime('%Y-%m-%d') if 'returns' in globals() else '2015-01-01'
end = returns.index[-1].strftime('%Y-%m-%d') if 'returns' in globals() else None

bil = yf.download('BIL', start=start, end=end, progress=False)['Adj Close'].dropna()
bil_returns = bil.pct_change().dropna()

# annualized realized risk-free rate over the period
rf_annual = (1 + bil_returns.mean())**252 - 1
rf_daily = bil_returns.mean()

print(f"Sample period: {start} to {end}")
print(f"Realized annualized risk-free rate (BIL proxy): {rf_annual:.4%}")

# Update Sharpe for portfolio using rf
try:
    ann_ret = (1 + portfolio_returns.mean())**252 - 1
    ann_vol = portfolio_returns.std(ddof=1) * (252**0.5)
    sharpe_real = (ann_ret - rf_annual) / ann_vol
    print(f"Portfolio annualized return: {ann_ret:.4%}, vol: {ann_vol:.4%}, Sharpe (BIL rf): {sharpe_real:.3f}")
except Exception as e:
    print('Could not compute portfolio Sharpe with BIL rf:', e)


---

##  Discussion Prompts & Teaching Notes

1. **t-VaR vs Gaussian VaR:** For the sample, which VaR is largest? Why? Discuss parameter sensitivity and sample size issues.  
2. **Hill plot selection:** Identify a stable region in the Hill plot. How would you choose `k` in practice? Consider bootstrap confidence bands.  
3. **Systematic vs Idiosyncratic:** If idiosyncratic variance dominates for AAPL, what does that imply about diversification strategies?  
4. **Risk-free proxy caveats:** Discuss when using an ETF proxy for the risk-free rate is reasonable and when it may be misleading (fees, tracking error, liquidity).

---
*Notebook updated on: 2025-10-15 22:51:12 UTC*
